# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 30 2022 8:41AM,247615,19,MSP214435,"Methapharm, Inc.",Released
1,Sep 30 2022 8:41AM,247615,19,ADV80006115,"AdvaGen Pharma, Ltd",Released
2,Sep 30 2022 8:41AM,247615,19,ADV80006116,"AdvaGen Pharma, Ltd",Released
3,Sep 30 2022 8:41AM,247615,19,ADV80006117,"AdvaGen Pharma, Ltd",Released
4,Sep 30 2022 8:41AM,247615,19,ADV80006120,"AdvaGen Pharma, Ltd",Released
5,Sep 30 2022 8:41AM,247615,19,ADV80006122,"AdvaGen Pharma, Ltd",Released
6,Sep 30 2022 8:41AM,247615,19,ADV80006123,"AdvaGen Pharma, Ltd",Released
7,Sep 30 2022 8:41AM,247615,19,ADV80006129,"AdvaGen Pharma, Ltd",Released
8,Sep 30 2022 8:41AM,247615,19,ADV80006130,"AdvaGen Pharma, Ltd",Released
9,Sep 30 2022 8:41AM,247615,19,ADV80006131,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
16,247607,Released,1
17,247608,Released,3
18,247612,Released,15
19,247614,Released,2
20,247615,Released,12


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247607,NaN,NaN,1.0
247608,NaN,NaN,3.0
247612,NaN,NaN,15.0
247614,NaN,NaN,2.0
247615,NaN,NaN,12.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247396,0.0,0.0,1.0
247534,0.0,0.0,1.0
247567,0.0,0.0,18.0
247576,0.0,0.0,31.0
247578,5.0,4.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247396,0,0,1
247534,0,0,1
247567,0,0,18
247576,0,0,31
247578,5,4,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247396,0,0,1
1,247534,0,0,1
2,247567,0,0,18
3,247576,0,0,31
4,247578,5,4,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247396,,,1
1,247534,,,1
2,247567,,,18
3,247576,,,31
4,247578,5,4,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 30 2022 8:41AM,247615,19,"Methapharm, Inc."
1,Sep 30 2022 8:41AM,247615,19,"AdvaGen Pharma, Ltd"
12,Sep 30 2022 8:30AM,247608,10,"Methapharm, Inc."
15,Sep 30 2022 8:30AM,247598,10,ISDIN Corporation
36,Sep 30 2022 8:30AM,247599,10,"Nextsource Biotechnology, LLC"
37,Sep 30 2022 8:30AM,247601,10,ISDIN Corporation
43,Sep 30 2022 8:30AM,247604,10,ISDIN Corporation
44,Sep 30 2022 8:03AM,247614,10,Emerginnova
46,Sep 29 2022 5:52PM,247612,15,"Alliance Pharma, Inc."
61,Sep 29 2022 5:27PM,247567,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 30 2022 8:41AM,247615,19,"Methapharm, Inc.",,,12
1,Sep 30 2022 8:41AM,247615,19,"AdvaGen Pharma, Ltd",,,12
2,Sep 30 2022 8:30AM,247608,10,"Methapharm, Inc.",,,3
3,Sep 30 2022 8:30AM,247598,10,ISDIN Corporation,,,21
4,Sep 30 2022 8:30AM,247599,10,"Nextsource Biotechnology, LLC",,,1
5,Sep 30 2022 8:30AM,247601,10,ISDIN Corporation,,,6
6,Sep 30 2022 8:30AM,247604,10,ISDIN Corporation,,,1
7,Sep 30 2022 8:03AM,247614,10,Emerginnova,,,2
8,Sep 29 2022 5:52PM,247612,15,"Alliance Pharma, Inc.",,,15
9,Sep 29 2022 5:27PM,247567,15,"Person & Covey, Inc.",,,18


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 30 2022 8:41AM,247615,19,"Methapharm, Inc.",12,,
1,Sep 30 2022 8:41AM,247615,19,"AdvaGen Pharma, Ltd",12,,
2,Sep 30 2022 8:30AM,247608,10,"Methapharm, Inc.",3,,
3,Sep 30 2022 8:30AM,247598,10,ISDIN Corporation,21,,
4,Sep 30 2022 8:30AM,247599,10,"Nextsource Biotechnology, LLC",1,,
5,Sep 30 2022 8:30AM,247601,10,ISDIN Corporation,6,,
6,Sep 30 2022 8:30AM,247604,10,ISDIN Corporation,1,,
7,Sep 30 2022 8:03AM,247614,10,Emerginnova,2,,
8,Sep 29 2022 5:52PM,247612,15,"Alliance Pharma, Inc.",15,,
9,Sep 29 2022 5:27PM,247567,15,"Person & Covey, Inc.",18,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 30 2022 8:41AM,247615,19,"Methapharm, Inc.",12,,
1,Sep 30 2022 8:41AM,247615,19,"AdvaGen Pharma, Ltd",12,,
2,Sep 30 2022 8:30AM,247608,10,"Methapharm, Inc.",3,,
3,Sep 30 2022 8:30AM,247598,10,ISDIN Corporation,21,,
4,Sep 30 2022 8:30AM,247599,10,"Nextsource Biotechnology, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 30 2022 8:41AM,247615,19,"Methapharm, Inc.",12.0,NaN,NaN
1,Sep 30 2022 8:41AM,247615,19,"AdvaGen Pharma, Ltd",12.0,NaN,NaN
2,Sep 30 2022 8:30AM,247608,10,"Methapharm, Inc.",3.0,NaN,NaN
3,Sep 30 2022 8:30AM,247598,10,ISDIN Corporation,21.0,NaN,NaN
4,Sep 30 2022 8:30AM,247599,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 30 2022 8:41AM,247615,19,"Methapharm, Inc.",12.0,0.0,0.0
1,Sep 30 2022 8:41AM,247615,19,"AdvaGen Pharma, Ltd",12.0,0.0,0.0
2,Sep 30 2022 8:30AM,247608,10,"Methapharm, Inc.",3.0,0.0,0.0
3,Sep 30 2022 8:30AM,247598,10,ISDIN Corporation,21.0,0.0,0.0
4,Sep 30 2022 8:30AM,247599,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1980778,67.0,4.0,5.0
15,495179,33.0,0.0,0.0
16,247534,1.0,0.0,0.0
19,742832,24.0,1.0,0.0
21,990214,3.0,1.0,0.0
22,495189,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1980778,67.0,4.0,5.0
1,15,495179,33.0,0.0,0.0
2,16,247534,1.0,0.0,0.0
3,19,742832,24.0,1.0,0.0
4,21,990214,3.0,1.0,0.0
5,22,495189,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,67.0,4.0,5.0
1,15,33.0,0.0,0.0
2,16,1.0,0.0,0.0
3,19,24.0,1.0,0.0
4,21,3.0,1.0,0.0
5,22,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,67.0
1,15,Released,33.0
2,16,Released,1.0
3,19,Released,24.0
4,21,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21,22
Status,,,,,,
Completed,5.0,0.0,0.0,0.0,0.0,0.0
Executing,4.0,0.0,0.0,1.0,1.0,1.0
Released,67.0,33.0,1.0,24.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21,22
0,Completed,5.0,0.0,0.0,0.0,0.0,0.0
1,Executing,4.0,0.0,0.0,1.0,1.0,1.0
2,Released,67.0,33.0,1.0,24.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21,22
0,Completed,5.0,0.0,0.0,0.0,0.0,0.0
1,Executing,4.0,0.0,0.0,1.0,1.0,1.0
2,Released,67.0,33.0,1.0,24.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()